# <div align="center" style="color: #ff5733;">New KPI Report</div>

## *Requestor is Jennyfer & Anne*

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# FPD30 July 2024 Month End

In [2]:
sq = """SELECT t1.new_loan_type,
ROUND(SUM(CASE when t1.defFPD30 = 1 then 1 ELSE 0 end )/sum(t1.obsFPD30)*100,3) as percentage_of_FPD30
from `prj-prod-dataplatform.risk_credit_mis.loan_master_table` t1
inner JOIN prj-prod-dataplatform.risk_credit_mis.loan_bucket_flow_report_core t2
ON
t1.loanAccountNumber = t2.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-07-31')
GROUP BY 1;"""

july24fpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
july24fpd30

Job ID 0247850f-60c8-4fde-8fc8-4b213667a98c successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,percentage_of_FPD30
0,Flex,0.820
1,SIL-Instore,6.629
2,Quick,8.236


# 30+, 60+, 90+ for July 2024

In [4]:
sq = """with b as
(select t1.new_loan_type 
, count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-07-31')
group by 1
)
select 
new_loan_type,
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
group by 1
order by 1
;"""

secondpartjuly24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
secondpartjuly24

Job ID 5a8aaec8-a2e3-4552-af38-ded83d065e3d successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,new_loan_type,thirtyplus,sixtyplus,nintyplus
0,Flex,35.363,31.030,26.171
1,Quick,46.535,39.227,43.690
2,SIL-Instore,21.953,15.989,20.882


# 30+, 60+, 90+ for July 2024 (without Product)

In [5]:
sq = """with b as
(select 
count(distinct t2.loanAccountNumber) cnt_loan
, count(distinct case when t2.Max_current_DPD >=30 then t2.loanAccountNumber end) thirtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=30 then t2.loanAccountNumber end) obsthirtyplus
, count(distinct case when t2.Max_current_DPD >=60 then t2.loanAccountNumber end) sixtyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >=60 then t2.loanAccountNumber end) obssixtyplus
, count(distinct case when t2.Max_current_DPD >=90 then t2.loanAccountNumber end) nintyplus
, count(distinct case when date_diff(date(t2.bucketDate), date(t2.firstduedate), day) >= 90 then t2.loanAccountNumber end) obsnintyplus
from `risk_credit_mis.loan_bucket_flow_report_core` t2
inner join `risk_credit_mis.loan_master_table` t1 on t2.loanAccountNumber = t1.loanAccountNumber
WHERE t1.flagDisbursement = 1
and t1.new_loan_type in ('Quick','Flex','SIL-Instore')
and t2.loanStatus in ('In Arrears','Normal')
and t2.bucketDate in ('2024-07-31')
)
select 
round(sum(thirtyplus)/sum(obsthirtyplus)*100, 3) thirtyplus,
round(sum(sixtyplus)/sum(obsthirtyplus)*100, 3) sixtyplus,
round(sum(nintyplus)/sum(obsnintyplus)*100, 3) nintyplus,
from b
order by 1
;"""

thirdpartjuly24 = client.query(sq).to_dataframe(progress_bar_type='tqdm')
thirdpartjuly24

Job ID 98edc136-9c2e-4430-a04b-40295eb92af9 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,thirtyplus,sixtyplus,nintyplus
0,25.596,19.514,24.962


In [6]:
dfjuly24 = july24fpd30.merge(secondpartjuly24, on='new_loan_type', how='left')
dfjuly24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Flex,0.820,35.363,31.030,26.171
1,SIL-Instore,6.629,21.953,15.989,20.882
2,Quick,8.236,46.535,39.227,43.690


In [7]:
resultjuly24 = pd.concat([dfjuly24, thirdpartjuly24], ignore_index=True)


,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Flex,0.820,35.363,31.030,26.171
1,SIL-Instore,6.629,21.953,15.989,20.882
2,Quick,8.236,46.535,39.227,43.690
3,NaN,NaN,25.596,19.514,24.962


In [8]:
resultjuly24['new_loan_type'] = resultjuly24['new_loan_type'].fillna("Portfolio")
resultjuly24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,Flex,0.820,35.363,31.030,26.171
1,SIL-Instore,6.629,21.953,15.989,20.882
2,Quick,8.236,46.535,39.227,43.690
3,Portfolio,NaN,25.596,19.514,24.962


In [9]:
import pandas as pd
from openpyxl import load_workbook


In [18]:
# Load the existing Excel file
existing_excel_file = r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\20240712_KRI_Report_New.xlsx'
xls = pd.ExcelFile(existing_excel_file)

# Read each existing sheet into separate dataframes
df_january = pd.read_excel(xls, sheet_name='January2024')
df_february = pd.read_excel(xls, sheet_name='February2024')
df_march = pd.read_excel(xls, sheet_name='March2024')
df_april = pd.read_excel(xls, sheet_name='April2024')
df_may = pd.read_excel(xls, sheet_name='May2024')
df_june = pd.read_excel(xls, sheet_name='June2024')




In [19]:
df_june

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,SIL-Instore,6.831,22.938,17.128,19.962
1,Quick,10.838,56.998,51.518,53.406
2,Flex,1.345,37.320,33.141,28.770
3,Portfolio,NaN,29.413,23.738,27.542


In [27]:
# Mapping function to modify 'new_loan_type' values
def modify_loan_type(loan_type):
    mapping = {'Flex': 'A. Flex', 'SIL-Instore': 'C. SIL-Instore', 'Quick': 'B. Quick', 'Portfolio': 'D. Portfolio'}
    return mapping.get(loan_type, loan_type)  # Return original value if not found

# Apply the mapping function to the 'new_loan_type' column
resultjuly24['new_loan_type'] = resultjuly24['new_loan_type'].apply(modify_loan_type)
resultjuly24

,new_loan_type,percentage_of_FPD30,thirtyplus,sixtyplus,nintyplus
0,A. Flex,0.820,35.363,31.030,26.171
1,C. SIL-Instore,6.629,21.953,15.989,20.882
2,B. Quick,8.236,46.535,39.227,43.690
3,D. Portfolio,NaN,25.596,19.514,24.962


In [30]:
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(r'C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Collection\Jennifer Zaldivar\KRI_New_Report\Data\20240802_KRI_Report_New.xlsx', engine='xlsxwriter')

# Sort each DataFrame by the first column and write to a separate worksheet, ignoring the index
df_january.sort_values(by=df_january.columns[0]).to_excel(writer, sheet_name='January2024', index=False)
df_february.sort_values(by=df_february.columns[0]).to_excel(writer, sheet_name='February2024', index=False)
df_march.sort_values(by=df_march.columns[0]).to_excel(writer, sheet_name='March2024', index=False)
df_april.sort_values(by=df_april.columns[0]).to_excel(writer, sheet_name='April2024', index=False)
df_may.sort_values(by=df_may.columns[0]).to_excel(writer, sheet_name='May2024', index=False)
df_june.sort_values(by=df_june.columns[0]).to_excel(writer, sheet_name='June2024', index=False)
resultjuly24.sort_values(by=resultjuly24.columns[0]).to_excel(writer, sheet_name='July2024', index=False)

# Close the Pandas Excel writer and output the Excel file
writer.close()